## **Library import** ##

In [ ]:
import requests
import time

In [ ]:
from dotenv import load_dotenv
import os

## **Data Collection - API** 📊

1. **Players** 🏀  
   - endpoint: https://api.balldontlie.io/v1/players
2. **Teams** 🏆  
   - endpoint: https://api.balldontlie.io/v1/teams


In [ ]:
load_dotenv()

In [ ]:
# Utilizo la clave API que se me ha otorgado al registrarme en la web.
api_key = 'xxxxx'
url = "https://api.balldontlie.io/v1/players"

### Incluyo los parametros que permiten la extracción de datos y la paginación.
headers = {"Authorization": api_key}
params = {
    "per_page": 100,
} # Le digo que me extraiga 100 jugadores por página que es el max en este caso.
### Creo diccionario para almacenar todos los jugadores registrados e incluyo la variable cursor que determina la página inicial.
cursor = None
todos_los_jugadores = []

while True:
    # Añadimos el cursor a los parámetros si existe
    if cursor:
        params["cursor"] = cursor
    
    # Realizar la solicitud GET sobre la API players
    response = requests.get(url, headers=headers, params=params)

    # Verifico el estado de la respuesta y si es igual a 200 procesa la request
    if response.status_code == 200:
        data = response.json()
        jugadores = data["data"]
        
        # Agrego los jugadores obtenidos a la lista
        todos_los_jugadores.extend(jugadores)
        
        # Actualizo el cursor para que se mueva a la próxima página
        cursor = data["meta"].get("next_cursor")
        
        # Rompe el bucle si no hay más páginas
        if not cursor:
            break
        time.sleep(2)
        # Incluyo un intervalo de 2 segundos por cada request ya que la API sólo permite 30 extracciones por minuto, de esta manera consigo extraer todos los jugadores posibles
    else:
        print(f"Error: {response.status_code}")
        break
        # Si surge algun error durante la extracción me devolverá un error y concluirá el bucle

print(f"Total de jugadores obtenidos: {len(todos_los_jugadores)}")


In [ ]:
api_key = 'xxxxx'
url = "https://api.balldontlie.io/v1/teams"

headers = {
    "Authorization": api_key
}
# En este caso no incluyo parámetros porque el total de equipos vienen en la primera página
response = requests.get(url, headers=headers)

if response.status_code == 200:
    data1 = response.json()
    equipos = data1["data"]
else:
    print(f"Error: {response.status_code}")

## **Conversion to data frames** ##

In [ ]:
for jugador in todos_los_jugadores:
    jugador['team'] = jugador['team']['id']
# realizo un bucle para cambiar la key team por el id que viene dentro como un subdiccionario

In [ ]:
todos_los_jugadores

In [ ]:
df_jugadores = pd.DataFrame(todos_los_jugadores)

In [ ]:
df_jugadores

In [ ]:
equipos

In [ ]:
df_equipos = pd.DataFrame(equipos)

In [ ]:
df_equipos

## **Saving data frames** ##

In [ ]:
df_jugadores.to_csv("api_players.csv", index=False)

In [ ]:
df_equipos.to_csv("api_teams.csv", index=False)